In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from imblearn.over_sampling import SMOTE
from collections import Counter
import tensorflow as tf
import numpy as np
tokenizer = Tokenizer(oov_token="<OOV>")
split = round(len(df)*0.8)
train_reviews = df['url'][:split]
train_label = df['label'][:split]
test_reviews = df['url'][split:]
test_label = df['label'][split:]
training_sentences = []
training_labels = []
testing_sentences = []
testing_labels = []
for row in train_reviews:
    training_sentences.append(str(row))
for row in train_label:
    training_labels.append(row)
for row in test_reviews:
    testing_sentences.append(str(row))
for row in test_label:
    testing_labels.append(row)
vocab_size = 400000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
oov_tok = '<OOV>'
padding_type = 'post'
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)
testing_sentences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sentences, maxlen=max_length)
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,       input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)
counter = Counter(training_labels_final)
smt = SMOTE()
X_train_sm,y_train_sm = smt.fit_resample(padded,training_labels_final)
num_epochs = 20
history = model.fit(X_train_sm,y_train_sm, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))
reccomendations = ("""Protections How to Protect Your Computer Below are some key steps to protecting your computer from intrusion:\n\n Keep Your Firewall Turned On: A firewall helps protect your computer from hackers who might try to gain access to crash it,\n
delete information, or even steal passwords or other sensitive information. Software firewalls are widely recommended for single\n
computers. The software is prepackaged on some operating systems or can be purchased for individual computers.\n
For multiple networked computers, hardware routers typically provide firewall protection.\n\n
Install or Update Your Antivirus Software: Antivirus software is designed to prevent malicious software programs from embedding on\n
your computer. If it detects malicious code, like a virus or a worm, it works to disarm or remove it. Viruses can infect computers\n
without users’ knowledge. Most types of antivirus software can be set up to update automatically.\n\n
Install or Update Your Antispyware Technology: Spyware is just what it sounds like—software that is surreptitiously installed on your\n
computer to let others peer into your activities on the computer. Some spyware collects information about you without your consent or\n
produces unwanted pop-up ads on your web browser. Some operating systems offer free spyware protection, and inexpensive software is\n
readily available for download on the Internet or at your local computer store. Be wary of ads on the Internet offering downloadable\n
antispyware—in some cases these products may be fake and may actually contain spyware or other malicious code.\n
It’s like buying groceries—shop where you trust.\n\n
Keep Your Operating System Up to Date: Computer operating systems are periodically updated to stay in tune with technology requirements\n
and to fix security holes. Be sure to install the updates to ensure your computer has the latest protection.\n\n
Be Careful What You Download: Carelessly downloading e-mail attachments can circumvent even the most vigilant anti-virus software.\n
Never open an e-mail attachment from someone you don’t know, and be wary of forwarded attachments from people you do know.\n
They may have unwittingly advanced malicious code.\n\n
Turn Off Your Computer: With the growth of high-speed Internet connections, many opt to leave their computers on and ready for action.\n
The downside is that being “always on” renders computers more susceptible. Beyond firewall protection, which is designed to fend off\n
unwanted attacks, turning the computer off effectively severs an attacker’s connection—be it spyware or a botnet that employs your\n
computer’s resources to reach out to other unwitting users.""")

print("Insert the Link")
data = input("Paste the Link Here\t")
print('Prediction Started ...')
t0= time.perf_counter()
data = str(data)
data = tokenizer.texts_to_sequences(data)
data = pad_sequences(data, maxlen=max_length)
score =model.predict(data).round(0).astype('int')
t1 = time.perf_counter() - t0
print('Prediction Completed\nTime taken',t1 ,'sec')
if score == 0:
  print("The URL is probaly a phising URL. Kindly read through the following Reccomendations\n",reccomendations)
else:
  print("The website is secure, kindly click the link to proceed")
  print(data)
classes = np.argmax(score, axis = 1)
print(classes)